In [1]:
import pandas as pd
import numpy as np

In [2]:
epi = pd.read_csv('data/EPI_data/ObservationData_rqridaf.csv')

In [3]:
demo = pd.read_csv('data/Demographic_data/demograpic_data.csv')

***Data Import***

In [90]:
def epi_to_country_data(country_name):
    country = epi[epi['location'] == country_name]

    indicators = list(country['indicator'].value_counts().to_dict().keys())

    min_date = country['Date'].min()
    max_date = country['Date'].max()

    dates = pd.Series(list(range(min_date, max_date + 1)))
    transpose_country = pd.DataFrame()
    transpose_country['date'] = dates
    transpose_country['country_name'] = country_name

    for indicator in indicators:
        indicator_df = country[country['indicator'] == indicator]

        date_values = {key: None for key in list(range(min_date, max_date + 1))}
        for row in indicator_df.iterrows():
            date_values[row[1]['Date']] = row[1]['Value']

        transpose_country[indicator] = date_values.values()
        
    return transpose_country

In [85]:
best_indicators = [key for key, value in epi['indicator'].value_counts().to_dict().items() if value > 500]

In [101]:
epi = epi[epi['indicator'].isin(best_indicators)]

In [102]:
epi_new = pd.DataFrame()

for country_name in  set(epi['location'].values):
    country_df = epi_to_country_data(country_name)
    epi_new = pd.concat([epi_new, country_df], ignore_index = True)

In [104]:
epi_new.head()

,date,country_name,Agriculture,Environmental Health,Terrestrial biome protection (national weights),Environmental Performance Index,Sanitation & Drinking Water,Biodiversity & Habitat,Unsafe sanitation,Ecosystem Vitality,...,Change in Forest Cover,Pesticide Regulation,Agricultural Subsidies,Change of Trend in Carbon Intensity,Climate,Marine Protected Areas,Fisheries,Fish Stock Status,Coastal Shelf Fishing Pressure,Critical Habitat Protection
0,2002,Hungary,46.45,83.63,29.57,67.51,96.24,29.57,99.99,56.76,...,100.0,84.0,8.9,44.03,66.87,NaN,NaN,NaN,NaN,NaN
1,2003,Hungary,46.73,84.75,29.57,67.97,99.63,29.57,99.99,56.78,...,100.0,84.0,9.46,44.03,66.87,NaN,NaN,NaN,NaN,NaN
2,2004,Hungary,48.88,85.39,29.57,68.3,99.99,29.57,99.99,56.9,...,100.0,84.0,13.75,44.03,66.87,NaN,NaN,NaN,NaN,NaN
3,2005,Hungary,51.65,85.45,29.57,68.41,99.99,29.57,99.99,57.05,...,100.0,84.0,19.31,44.03,66.87,NaN,NaN,NaN,NaN,NaN
4,2006,Hungary,52.7,85.12,29.57,68.31,99.99,29.57,99.99,57.11,...,100.0,84.0,21.41,44.03,66.87,NaN,NaN,NaN,NaN,NaN
